# Pase each page of Makeup product

In [20]:
import re
import os
import time
import random
import warnings
import pandas as pd
import numpy as np
from typing import List
from lxml import etree as et
from bs4 import BeautifulSoup
from time import sleep
from ediblepickle import checkpoint
from urllib.parse import quote

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

opt=Options()
opt.add_argument('--no-sandbox')   
opt.add_argument('--disable-dev-shm-usage')
opt.add_argument('--disable-gpu')
warnings.filterwarnings('ignore')
#driver = webdriver.Chrome(ChromeDriverManager().install())
#driver = webdriver.Chrome(chrome_options=opt)

In [19]:
mu=pd.read_csv('Makeup.csv')
#split reviews and ratings
mu[['Rating','Reviews']]=mu['Rating'].str.split(';',expand=True)
mu['Rating']=[str(x).split(" ")[0] for x in mu['Rating']]
mu['Reviews']=[str(x).lstrip().split(" ")[0]for x in mu['Reviews']]
mu['Ingredient']=""
mu['Category']=''
mu['Details']=''
mu['Image']=''
mu.sample(3)

,Brand,Name,Rating,Price,Link,ID,Reviews,Ingredient,Category,Details,Image
3187,e.l.f. Cosmetics,Illuminating Mist & Set,3.9,$6.00,https://www.ulta.com/p/illuminating-mist-set-p...,2520970,90,,,,
993,KYLIE COSMETICS,Pressed Bronzing Powder,4.2,$19.00,https://www.ulta.com/p/pressed-bronzing-powder...,2584043,217,,,,
2555,Grande Cosmetics,Most Loved Nudes 2.0 Set,4.6,$27.00,https://www.ulta.com/p/most-loved-nudes-20-set...,2596476,8,,,,


In [45]:
def get_ingredients():
    try:                                                                                
        driver.find_element("xpath",'//*[@id="Ingredients"]').click()
        time.sleep(1)               
        ing = driver.find_element("xpath", "//*[@aria-controls='Ingredients']").text
        if ing:
            return re.sub(r'[\[\]\']', '', str(ing))
        else:
            return "Ingredients are not available"
    except:                                                                              
        return "Ingredients are not available"


In [46]:
#Fetch product pic link
def get_pic_link(index):
    return "https://media.ulta.com/i/ulta/{}?w=582&h=582&fmt=webp".format(index)

#get_pic_link(2309445)

In [47]:
#Fetching product details
def get_detail():    
    try:                                                                                
        driver.find_element("xpath",'//*[@id="Details"]').click()
        time.sleep(1)               
        details = driver.find_element("xpath", "//*[@aria-controls='Details']").text
        if details:
            return details
        else:
            return "Details are not available"
    except:                                                                              
        return "Details are not available"


In [48]:
#Fetch product category
def get_category():
    try:
        cat_list=driver.find_element("xpath", "//*[@id='Breadcrumbs__List']/li[3]/a").text
        return cat_list.strip()
    except Exception:
        return "Category is not available"

#get_category()

In [57]:
%%time
#Parse each product page, fetch Ingredients, Category,Detail

cache_dir = 'cache_2'
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)


@checkpoint(key=lambda args, kwargs: quote(str(args[1])) + '.pkl', work_dir=cache_dir)
def parse_page(df,index):
    
    prod_link=df['Link'][index]                
    driver.get(prod_link)
    sleep(1)
    #sleep(np.random.randint(2,3))
    df['Ingredient'][index]=get_ingredients()
    df['Category'][index]=get_category()
    df['Details'][index]=get_detail()
    df['Image'][index]=get_pic_link(index)
    
    #print(index)
    if(index%200==0):
        mu.to_csv('ProdDetail/mu{}.csv'.format(i),index=None)
    return

CPU times: user 172 µs, sys: 94 µs, total: 266 µs
Wall time: 237 µs


In [60]:
%%time

driver = webdriver.Chrome(chrome_options=opt)
for i in range(len(mu)):
    try:
        parse_page(mu,i)
    except Exception:
        driver.quit()
        driver = webdriver.Chrome(chrome_options=opt)
        parse_page(mu,i)
        print("Exceptioin: "+str(i))
        continue


Exceptioin: 4088
Exceptioin: 4491
CPU times: user 21 s, sys: 2.55 s, total: 23.6 s
Wall time: 3h 39min 1s


In [63]:
mu.to_csv('MakeupDetail.csv',index=None)
mu.tail(5)

,Brand,Name,Rating,Price,Link,ID,Reviews,Ingredient,Category,Details,Image
4891,Smashbox,Cover Shot Eyeshadow Palette: Matte,4.4,$32.00,https://www.ulta.com/p/cover-shot-eyeshadow-pa...,2503595,646,Ingredients are not available,Category is not available,Details are not available,https://media.ulta.com/i/ulta/4891?w=582&h=582...
4892,NARS,Smudge Proof Eyeshadow Base,4.6,$26.00,https://www.ulta.com/p/smudge-proof-eyeshadow-...,2511521,93,Ingredients are not available,Category is not available,Details are not available,https://media.ulta.com/i/ulta/4892?w=582&h=582...
4893,FLOWER Beauty,Miracle Matte Translucent Finishing Powder,3.3,$13.99,https://www.ulta.com/p/miracle-matte-transluce...,2524919,245,Ingredients are not available,Category is not available,Details are not available,https://media.ulta.com/i/ulta/4893?w=582&h=582...
4894,CoverGirl,Flourish by Lash Blast Mascara,4.3,$10.99,https://www.ulta.com/p/flourish-by-lash-blast-...,2531617,1897,Ingredients are not available,Category is not available,Details are not available,https://media.ulta.com/i/ulta/4894?w=582&h=582...
4895,NYX Professional Makeup,Swear By It Jumbo Eyeshadow Palette,4.5,$35.00,https://www.ulta.com/p/swear-by-it-jumbo-eyesh...,2533282,96,Ingredients are not available,Category is not available,Details are not available,https://media.ulta.com/i/ulta/4895?w=582&h=582...


In [62]:
driver.quit()